In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
import gensim
#from spellchecker import SpellChecker

# To display full text
pd.set_option('display.max_colwidth', -1)

# Load the Lemmatisation function --------------------------------------------------------
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


# Load the Stop words ---------------------------------------------------------------------
from sklearn.feature_extraction.text import CountVectorizer
 
# Defining custom stop words
stopwords = nltk.corpus.stopwords.words('english')
stopwords = stopwords  + ['cup','teaspoon','tablespoon','sweet', 'low', 'high','medium', 'chopped', 'crushed', 'pound', 'small', 'fresh', 'clove', 'oz', 
                'ounce','cut','taste','thinly','lengthwis','extra','garnish','finely','long','short','inch','thin','pieces',
                'wide','lightly','country','discarded','across','package','packed','pieces','extra','squeezed','sometimes',
                'half','free','box','container','jar','equipment','pale','lengthwise','perferably','note','divided','piece',
                'part','separated','bunch','large','lb','kosher','salt','freshly_ground','ground','plus_more','minced',
                'cut_into','peeled','thinly_sliced','tbsp','tsp','gram','dice','room_temperature','coarsely','coarse_kosher','coarse','dash_of',
                'ml','plus','inch_cubes','water','diced','seeded','dried','frozen','such_as','red','white','green','brown','oil','chilled',
                'grated','slice','sliced','thick','to_taste','leaf','ounce_can','peel','hot','cold','wedge','pinch_of',
                'whole','size','remove','removed','strip','special_equipment','optional','trimmed','crosswise','xa','fine','sliced_thin',
                'halved','halved_lengthwise','quartered','inch_thick_slices','crumbled','inch_dice','drained','loosely_packed',
                'preferably','cooked','uncooked','thawed','pitted','cored','canned_low','powder','stick','round','tied',
                'dry_white','softened','soft','head','some_supermarkets','one','two','eight','add','grade','torn','left_intact','dash',
                'deveined','freshly','melted','for_garnish','strong','ounce bag','ounce_package','several','by_inch','quart','quarter',
                'lightly_beaten','if_needed','packed_golden_brown','picked','pale_green_parts_only','prepared','double','mild',
                'slivered','special_equipment_an_instant','at_room_temperature','medium','plus_more_if_needed','medium_size',
                'very_thinly_sliced','wear_rubber_gloves']

C:\Users\User\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# Pre Processing function -------------------------------------------------------
import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

In [3]:
# Define a function to calculate cosine similarity
import numpy.linalg as LA
cx = lambda a, b : round(np.inner(a, b)/(LA.norm(a)*LA.norm(b)), 3)

In [4]:
# Reading the data
import pandas as pd
recipes = pd.read_json('full_format_recipes.json')

In [5]:
# remove records with missing title
# 19 recipes removed
# recipes_final = recipes[recipes["ingredients"].notnull()]
recipes_final=recipes.dropna(subset=["title"], inplace=False)
recipes_final.reset_index(drop=True, inplace=True)
print(recipes_final.shape)
recipes_final.columns

(20111, 11)


Index(['calories', 'categories', 'date', 'desc', 'directions', 'fat',
       'ingredients', 'protein', 'rating', 'sodium', 'title'],
      dtype='object')

In [6]:
# remove duplicate recipes
recipes_final=pd.DataFrame(recipes_final)
#recipes_final.drop_duplicates(subset="title", keep='first', inplace=True)
recipes_final = recipes_final.iloc[recipes_final.astype(str).drop_duplicates().index]
recipes_final.reset_index(drop=True, inplace=True)

print(recipes_final.shape)
recipes_final.columns

(18295, 11)


Index(['calories', 'categories', 'date', 'desc', 'directions', 'fat',
       'ingredients', 'protein', 'rating', 'sodium', 'title'],
      dtype='object')

In [7]:
# combine "title" and "ingredients" as one string variable
recipes_final.iloc[:,6] = [str(ingrd) for ingrd in (recipes_final.iloc[:,6])]
recipes_final.iloc[:,10] = [str(title) for title in (recipes_final.iloc[:,10])]

recipes_final["titleingrd"]=recipes_final["title"]+recipes_final["ingredients"]

In [8]:
# Make sure the time format is correct
# We will consider only hours and minutes in the time component
recipes_final.date = recipes_final.date.dt.strftime('%m/%d/%Y %H:%M')

In [9]:
# Apply the above function to preprocess ingredients
recipes_final['ingredients_processed'] = recipes_final['titleingrd'].apply(lambda x:pre_process(x))

In [10]:
recipes_final.columns
recipes_final.iloc[1,12]

'boudin blanc terrine with red onion confit cups whipping cream medium onions chopped teaspoons salt bay leaves whole cloves large garlic clove crushed teaspoon pepper teaspoon ground nutmeg pinch of dried thyme crumbled large shallots minced tablespoon butter pound trimmed boneless center pork loin sinew removed cut into inch chunks well chilled eggs tablespoon all purpose flour cup tawny port tablespoons dried currants minced lettuce leaves cracked peppercorns minced fresh parsley bay leaves french bread baguette slices tablespoons olive oil large red onions halved sliced tablespoons dried currants tablespoons red wine vinegar tablespoons canned chicken broth teaspoons chopped fresh thyme or teaspoon dried crumbled teaspoon sugar '

In [11]:
# Define the sentence to be lemmatized
sentence = recipes_final.iloc[:,12]

# Perform lemmatization for each recipe (row)
lemmatized_output = []
for rcp in sentence:
    word_list = nltk.word_tokenize(rcp)
    lo = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    lemmatized_output.append(lo)

In [12]:
# Checking lemmatization
print("before:",sentence[10])
print("after:",lemmatized_output[10])

before: yams braised with cream rosemary and nutmeg teaspoons olive oil cup finely chopped shallots teaspoons minced fresh rosemary pounds yams red skinned sweet potatoes peeled cut into inch thick rounds rounds cut in half cups canned low salt chicken broth cup whipping cream ground nutmeg 
after: yam braised with cream rosemary and nutmeg teaspoon olive oil cup finely chopped shallot teaspoon minced fresh rosemary pound yam red skinned sweet potato peeled cut into inch thick round round cut in half cup canned low salt chicken broth cup whipping cream ground nutmeg


In [13]:
lem_out_wordlist = [nltk.word_tokenize(x) for x in lemmatized_output]

In [14]:
# Create the word count vector ------------------------------------------------------
# Here the stopwords are removed, and 
#ignore words that appear in 85% of documents and below 0.01%
from sklearn.feature_extraction.text import TfidfVectorizer

# cv=CountVectorizer(stop_words=stopwords,max_df=0.85,min_df= 0.0001)
# word_count_vector=cv.fit_transform(lemmatized_output)

TfidfVec = TfidfVectorizer(stop_words=stopwords,max_df=0.85,min_df= 0.0001,ngram_range=(1,2))
word_count_vector = TfidfVec.fit_transform(lemmatized_output)

# Convert to an array
trainVectorizerArray = word_count_vector.toarray()

In [15]:
# Subset columns to display
recipes_list = recipes_final[[ 'title','date','rating','calories', 'ingredients', 'directions', 'fat',
        'protein','sodium']]

# Reset index name 
recipes_list = pd.concat([pd.DataFrame(list(recipes_list.index)),recipes_list], axis=1)

# Rename the columns
recipes_list.columns = [ 'row_num' ,'title','date','rating','calories', 'ingredients', 'directions', 'fat',
        'protein','sodium']

In [16]:
# Function to retrieve recipes using the cosine distance between them

def getRecipe_cosine(query = "", sort = True):
    
    if query == "":
        return(print('no match found '))
    
    # Define empty arrays
    distance = [] # Distance between document
    row_num = [] # Row number of the document 
    
    # Pre-process query
    query_processed = pre_process(str(query))    
#     # Spell check the query
#     spell = SpellChecker()
#     misspelled = spell.unknown(query_processed)
    
#     corrected =""
#     spellcheck_query = ""
    
#     for word in misspelled:
#         print(spell.correction(word))
#         corrected=[spell.correction(word)]
#         if len(corrected)==0:
#             corrected.remove('non') 
#         #print(corrected)

#         spellcheck_query = str(query_processed) + str(corrected)

        
    # Lemmatize the query
    query_wlist = nltk.word_tokenize(query_processed)
    lemmatized_query = ' '.join([lemmatizer.lemmatize(w) for w in query_wlist])
    
    # Vectorize the words and convert to an array
    testVectorizerArray = TfidfVec.transform([lemmatized_query]).toarray()
    
    for x in range(len(trainVectorizerArray)):
        vector = trainVectorizerArray[x]
        for testV in testVectorizerArray:
            cosine = cx(vector, testV)
            if cosine > 0:
                distance.append(cosine)
                row_num.append(x)
    
    
    # Concatenate the columns into a dataframe
    matches = pd.concat([pd.DataFrame(distance),pd.DataFrame(row_num)],axis = 1)
    if matches.empty:
        return(print(" No Recipes with "+ query))
# Provide descriptive the column names
    matches.columns = ['similarity_val','row_num']
# Sort by the highest similarity value
    matches = matches.sort_values(by=['similarity_val'],ascending=False)
    print(matches.shape[0], "recipes matched")
    
    # Save the top 10 in a dataframe
    results = pd.DataFrame(recipes_list.iloc[matches.row_num[:10],:])
    results_score = pd.merge(results,matches[:10], how = 'left', on = 'row_num')
    
    if(sort):
        # Sort by time
        results = results.sort_values(by = ['date'], ascending=False)
        # sort my ratings
        results = results.sort_values(by = ['rating'], ascending=False)
    else:
        results = pd.DataFrame(recipes_list.iloc[matches.row_num[0:10],:])
        results_score = pd.merge(results,matches[0:10], how = 'left', on = 'row_num')
        #results_score = results_score.sort_values(by = ['similarity_val'],ascending=False)
        #results.to_excel('query_results_co.xlsx')
        
    #Return the dataset
    if query != '':
        return(results_score)

# Recipe Search

In [17]:
# To display full text
pd.set_option('display.max_colwidth', -1)

In [21]:
# Try the retrieval for a few test queries
query = input("Enter your ingredients search here  ")

Enter your ingredients search here  fish, chilli and pasta


In [22]:
%%time
getRecipe_cosine(query,sort = False)

960 recipes matched
Wall time: 3.24 s


,row_num,title,date,rating,calories,ingredients,directions,fat,protein,sodium,similarity_val
0,4595,Chilli Oil,03/14/2014 04:00,0.000,411.0,"['600ml (1 pint) rapeseed oil', '1 large red chilli, split in half', '1 lemon grass stalk', '1 garlic clove', '20g (3/4oz) root ginger, sliced but not peeled']","[Gently warm the oil through in a heavy-based pan but do not allow it to boil. Bring to a gentle simmer, then add the chilli, lemon grass, garlic and ginger. Continue to simmer very gently for 20–30 minutes, until the flavours are well infused. It's important not to allow it to boil at any stage. Pour into a squeezy bottle, leaving the bits in, as the flavours will continue to infuse. Use as required.]",46.0,0.0,1.0,0.427
1,15244,Sweet Potato and Coconut Soup,03/14/2014 04:00,4.375,NaN,"['450g (1lb) sweet potatoes, cut into cubes', '2 tbsp sunflower oil', '1 onion, finely chopped', '1 leek, trimmed and finely chopped', '1 lemon grass stalk, trimmed and halved', '1 red chilli, halved, seeded and thinly sliced', '1 tsp freshly grated root ginger', '1.25 litres (2 1/4 pints) vegetable stock', '4 tsp tomato purée', '400g can coconut milk', 'sea salt and freshly ground black pepper', 'chilli oil , to garnish', 'fresh micro coriander, to garnish']","[Preheat the oven to 200°C (400°F/gas mark 6)., Place the sweet potatoes in a baking tin, drizzle over 1 tablespoon of the sunflower oil and roast for 20–30 minutes, until tender. Set aside., Heat the remaining 1 tablespoon of oil in a pan over a medium heat. Add the onion, leek, lemon grass, half the chilli and the ginger and sweat for 4 minutes, stirring occasionally. Add the roasted sweet potato with the stock and tomato purée, then bring to the boil. Reduce the heat and simmer for 10 minutes, until the liquid has slightly reduced and all the vegetables are completely tender., Pour the coconut milk into the pan, reserving about 3 tablespoons from the top of the can as a garnish, and cook for another 5 minutes, stirring constantly. Season to taste. Remove the lemon grass and then blend with a hand blender until smooth., To serve, ladle the soup into warmed bowls and swirl in the reserved coconut milk. Add a drizzle of the chilli oil, then sprinkle over the reserved chilli slices and micro coriander.]",NaN,NaN,NaN,0.286
2,3401,Quince Sambal,05/16/2006 20:12,2.500,NaN,"['1 ripe quince', 'salt', '1 small onion, grated', '2 ml (1/2 teaspoon) crushed garlic', '1 small red or green chilli, sliced, seeded and finely chopped', '30 ml (2 tablespoons) sugar', '30 ml (2 tablespoons) lemon juice']","[Peel and core the quince, and grate coarsely or slice into the finest slivers. Pile in a bowl, sprinkle with salt and set aside for 1-2 hours. Rinse under cold running water, drain well, and dry thoroughly with a clean tea towel. Mix with the remaining ingredients, cover and chill until required.]",NaN,NaN,NaN,0.268
3,13461,Dukkah-Crusted Salmon With Cucumber and Chilli Salad,09/01/2016 19:14,1.875,NaN,"['1 1/2 cups (75g) puffed amaranth', '2 tablespoons store-bought dukkah', '1 teaspoon sea salt flakes', '4 (200g) salmon fillets, skin removed', '2 eggs, lightly beaten', '2 tablespoons extra-virgin olive oil', '1 long green chilli, thinly sliced', '1/4 cup (60ml) extra virgin olive oil, plus more', '1/4 cup (60ml) lime juice', '1 clove garlic, crushed', '2 tablespoons chopped cilantro', 'Sea salt and cracked black pepper', '4 cups (50g) snow pea tendrils', '2 Lebanese cucumbers (260g), thinly sliced', 'Chervil sprigs, to serve']","[Place the amaranth, dukkah and salt on a small tray and toss to combine. Dip each salmon fillet in the egg and press into the dukkah mixture to coat. Heat the oil in a large non-stick frying pan over medium heat. Cook the salmon, turning every 3–4 minutes, for 10–12 minutes or until just cooked through and the crumb is golden. Place the chilli, extra oil, lime juice, garlic, cilantro, salt and pepper in a medium bowl and whisk to combine. Add the snow pea tendrils and cucumber

# Evaluation of Document Retrieval 

### Precision & Recall

Precision measures "of all the documents we retrieved as relevant how many are actually relevant?"
Recall measures "Of all the actual relevant documents how many did we retrieve as relevant?"
F-Score / F-measure is the weighted harmonic mean of precision and recall. The traditional F-measure or balanced F-score is:

In [117]:
# Precision = No. of relevant documents retrieved / No. of total documents retrieved
precision = 6/10
print ("Precision =", precision)

# Recall = No. of relevant documents retrieved / No. of total relevant documents
recall = 6/20 # orbitary number here
print ("Recall = ", recall)

Fscore = 2 * precision * recall /(precision + recall)
print ("F-Score =", Fscore)

Precision = 0.6
Recall =  0.3
F-Score = 0.4
